In [134]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm

In [4]:
engine = create_engine("postgresql:///Seattle_housing")

In [23]:
query1 = """SELECT 
               CONCAT(rp."Major", rp."Minor") AS pin,
               "SalePrice", 
               "SqFtTotLiving" AS SquareFeet, 
               CASE 
                   WHEN ("SqFtOpenPorch" + "SqFtEnclosedPorch" > 0) THEN 1
                   ELSE 0 
                   END AS porch,
               CASE 
                   WHEN "WfntLocation" > 0 THEN 1
                   ELSE 0
                   END AS WaterFront,
               (("SqFt1stFloor" + "SqFtGarageAttached") / "SqFtLot" ) AS proportion,
               CASE 
                   WHEN ("TrafficNoise" + "AirportNoise" > 0) OR ("PowerLines" = 'Y') OR ("OtherNuisances" = 'Y') THEN 1
                   ELSE 0
                   END AS nuisance
            FROM rpsale rp
            JOIN resbldg rd
            ON CONCAT(rp."Major", rp."Minor") = Concat(rd."Major", rd."Minor")
            JOIN parcel p
            ON CONCAT(rp."Major", rp."Minor") = Concat(p."Major", p."Minor")
            WHERE "SalePrice" > 0
            AND SUBSTRING("DocumentDate", 7, 4) = '2018'
        """

In [26]:
df1 = pd.read_sql(sql = query1, con = engine)

In [27]:
query2 = """SELECT 
               *
            FROM rpsale rp
            JOIN resbldg rd
            ON CONCAT(rp."Major", rp."Minor") = Concat(rd."Major", rd."Minor")
            JOIN parcel p
            ON CONCAT(rp."Major", rp."Minor") = Concat(p."Major", p."Minor")
            WHERE "SalePrice" > 0
            AND SUBSTRING("DocumentDate", 7, 4) = '2018'
         """

In [44]:
seattle_housing = pd.read_sql(sql = query2, con = engine)

In [48]:
seattle_housing.columns

Index(['ExciseTaxNbr', 'Major', 'Minor', 'DocumentDate', 'SalePrice',
       'RecordingNbr', 'Volume', 'Page', 'PlatNbr', 'PlatType',
       ...
       'SeismicHazard', 'LandslideHazard', 'SteepSlopeHazard', 'Stream',
       'Wetland', 'SpeciesOfConcern', 'SensitiveAreaTract', 'WaterProblems',
       'TranspConcurrency', 'OtherProblems'],
      dtype='object', length=155)

In [56]:
seattle_housing.describe()

,ExciseTaxNbr,SalePrice,PropertyType,PrincipalUse,SaleInstrument,SaleReason,PropertyClass,BldgNbr,NbrLivingUnits,Stories,...,WfntPoorQuality,WfntRestrictedAccess,TidelandShoreland,LotDepthFactor,TrafficNoise,AirportNoise,NbrBldgSites,Contamination,HistoricSite,CurrentUseDesignation
count,3.027200e+04,3.027200e+04,30272.000000,30272.000000,30272.000000,30272.000000,30272.000000,30272.000000,30272.000000,30272.000000,...,30272.000000,30272.000000,30272.000000,30272.000000,30272.000000,30272.0,30272.000000,30272.0,30272.000000,30272.000000
mean,2.939931e+06,8.241142e+05,7.204975,6.000892,3.875496,2.073533,7.958476,1.020448,1.019094,1.584699,...,0.000925,0.006276,0.016286,0.026328,0.210128,0.0,0.016781,0.0,0.001255,0.006045
std,1.684523e+04,7.799482e+05,6.618774,0.104565,3.815973,3.999960,0.289602,0.253439,0.159175,0.561174,...,0.030399,0.117342,0.177987,1.388876,0.564682,0.0,0.398847,0.0,0.060814,0.127991
min,2.909224e+06,1.000000e+00,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
25%,2.925780e+06,4.408750e+05,3.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
50%,2.939630e+06,6.600000e+05,10.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
75%,2.954244e+06,9.361250e+05,11.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
max,3.017086e+06,2.675000e+07,99.000000,10.000000,28.000000,19.000000,9.000000,12.000000,3.000000,4.000000,...,1.000000,3.000000,2.000000,96.000000,3.000000,0.0,20.000000,0.0,3.000000,4.000000


In [37]:
# delcolumns = ["PropName", "PlatName", "PlatLot", "PlatBlock", "Range",
#               "Township", "Section", "QuarterSection", "Area", "SubArea",
#               "SpecArea", "SpecSubArea", "ExciseTaxNbr", "RecordingNbr",
#               "Volume", "Page", "PlatNbr", "PlatType",
#               "SellerName", "BuyerName", "BldgNbr", "Address", "BuildingNumber",
#               "Fraction", "DirectionPrefix", "StreetName", "StreetType", 
#               "DirectionSuffix", "AirportNoise", "Contamination"]

In [40]:
# for column in delcolumns:
#     del seattle_housing[column]

In [41]:
seattle_housing.head()

,Major,Minor,DocumentDate,SalePrice,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
0,000100,0066,01/29/2018,310000.0,3.0,6.0,3.0,N,N,N,...,N,N,N,N,N,N,N,N,N,N
1,000180,0081,07/31/2018,500000.0,3.0,6.0,3.0,N,N,N,...,N,N,N,N,N,N,N,N,N,N
2,000280,0024,07/17/2018,456800.0,11.0,6.0,3.0,N,N,N,...,N,N,N,N,N,N,N,N,N,N
3,000300,0031,12/20/2018,461000.0,11.0,6.0,3.0,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,000300,0059,03/28/2018,80000.0,1.0,6.0,3.0,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [168]:
seattle_housing['ZipCode'].unique()

array(['98002     ', '98108     ', '98168     ', '98178     ',
       '98022     ', '98032     ', '          ', '98057     ',
       '98055     ', '98122     ', '98001     ', '98034     ',
       '98052     ', '98042     ', '98040     ', '98136     ',
       '98126     ', '98146     ', '98188     ', '98023     ',
       '98058     ', '98116     ', '98030     ', '98027     ',
       '98103     ', '98075     ', '98024     ', '98003     ',
       '98031     ', '98115     ', '98117     ', '98177     ',
       '98155     ', '98019     ', '98028     ', '98033     ',
       '98092     ', '98070     ', '98107     ', '98106     ',
       '98059     ', '98065     ', '98008     ', '98133     ',
       '98072     ', '98077     ', '98125     ', '98102     ',
       '98056     ', '98068     ', '98045     ', '98148     ',
       '98011     ', '98005     ', '98053     ', '98014     ',
       '98029     ', '98038     ', '98118     ', '98007     ',
       '98075-8010', '98224     ', '98166     ', '98004

In [97]:
keepcol = ['SalePrice', 'SqFtTotLiving', 'SqFtLot', 'Bedrooms', 'BathHalfCount', 'Bath3qtrCount',
           'BathFullCount', 'YrBuilt', 'SqFtGarageBasement', 'SqFtGarageAttached', 'Area']

In [98]:
model1df = seattle_housing[keepcol]

In [99]:
#Make total number of bathrooms column - maybe consider multiplying by 0.5/0.75
model1df['Bathrooms'] = model1df['BathHalfCount'] + model1df['Bath3qtrCount'] + model1df['BathFullCount']

/opt/anaconda3/envs/linreg-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [100]:
model1df['AgeOfBuilding'] = 2018 - model1df['YrBuilt']

/opt/anaconda3/envs/linreg-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [101]:
model1df['Garage'] = model1df['SqFtGarageAttached'] + model1df['SqFtGarageBasement']

/opt/anaconda3/envs/linreg-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
model1df.head()

,SalePrice,SqFtTotLiving,SqFtLot,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,YrBuilt,SqFtGarageBasement,SqFtGarageAttached,Area,Bathrooms,AgeOfBuilding,Garage
0,310000.0,1200.0,14500.0,3.0,1.0,0.0,1.0,1960.0,0.0,600.0,28,2.0,58.0,600.0
1,500000.0,4100.0,16406.0,4.0,1.0,0.0,1.0,1961.0,240.0,0.0,79,2.0,57.0,240.0
2,456800.0,1720.0,8620.0,3.0,0.0,1.0,1.0,1948.0,0.0,0.0,24,2.0,70.0,0.0
3,461000.0,2840.0,8728.0,5.0,0.0,1.0,2.0,2000.0,600.0,0.0,24,3.0,18.0,600.0
4,80000.0,2980.0,12400.0,4.0,0.0,1.0,2.0,2019.0,0.0,660.0,24,3.0,-1.0,660.0


In [107]:
ohe = OneHotEncoder()

model1df_trans = ohe.fit_transform(model1df[['Area']])

/opt/anaconda3/envs/linreg-env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [113]:
model1df_trans.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]])

In [114]:
df = pd.DataFrame(model1df_trans.todense(), columns = ohe.get_feature_names())
df.head()

,x0_1.0,x0_2.0,x0_3.0,x0_4.0,x0_6.0,x0_7.0,x0_8.0,x0_11.0,x0_12.0,x0_13.0,...,x0_87.0,x0_88.0,x0_90.0,x0_91.0,x0_92.0,x0_93.0,x0_94.0,x0_95.0,x0_96.0,x0_100.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
model1_edit = pd.concat([model1df, df], axis=1)

In [159]:
#Remove house sales less than $5000
model1_edit = model1_edit[model1_edit['SalePrice'] >= 5000]

In [160]:
model1_edit = model1_edit.drop(['BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'YrBuilt', 'SqFtGarageBasement', 
                  'SqFtGarageAttached', 'Area', 'x0_100.0'], axis = 1)






In [161]:
model1_edit.corr()

,SalePrice,SqFtTotLiving,SqFtLot,Bedrooms,Bathrooms,AgeOfBuilding,Garage,x0_1.0,x0_2.0,x0_3.0,...,x0_86.0,x0_87.0,x0_88.0,x0_90.0,x0_91.0,x0_92.0,x0_93.0,x0_94.0,x0_95.0,x0_96.0
SalePrice,1.000000,0.510795,0.040651,0.236686,0.407119,-0.142082,0.183642,0.004859,-0.020984,-0.034021,...,-0.067848,-0.054578,-0.064829,-0.020806,0.000407,0.136023,0.026866,-0.020643,0.016523,-0.061861
SqFtTotLiving,0.510795,1.000000,0.114822,0.626467,0.777804,-0.374247,0.523475,0.014859,-0.040699,-0.055531,...,-0.022589,-0.021384,-0.050536,-0.031246,0.008513,0.112976,0.022995,0.001966,0.040474,-0.061492
SqFtLot,0.040651,0.114822,1.000000,-0.005922,0.044485,0.002403,0.077098,0.001143,-0.012947,-0.014096,...,-0.016625,-0.003258,-0.005169,0.004225,-0.010405,-0.005507,-0.014213,0.117909,0.062554,-0.014914
Bedrooms,0.236686,0.626467,-0.005922,1.000000,0.567140,-0.224351,0.321559,0.007384,-0.015431,-0.029904,...,0.017346,-0.007207,-0.020201,-0.038076,0.025598,0.054777,0.024964,-0.011521,-0.040870,-0.026024
Bathrooms,0.407119,0.777804,0.044485,0.567140,1.000000,-0.532041,0.506637,-0.004676,-0.040854,-0.066852,...,-0.014804,-0.007925,-0.049927,-0.033645,0.015978,0.092709,0.022038,-0.008664,0.021984,-0.077173
AgeOfBuilding,-0.142082,-0.374247,0.002403,-0.224351,-0.532041,1.000000,-0.496575,0.037041,0.032834,0.054898,...,-0.055574,-0.036677,0.024397,0.003111,0.006057,-0.021558,-0.042981,-0.014835,-0.049404,0.050488
Garage,0.183642,0.523475,0.077098,0.321559,0.506637,-0.496575,1.000000,0.005325,-0.034033,-0.052701,...,0.042104,0.015177,-0.013820,-0.025895,0.038197,0.053689,0.038843,0.014553,0.077333,-0.056018
x0_1.0,0.004859,0.014859,0.001143,0.007384,-0.004676,0.037041,0.005325,1.000000,-0.007877,-0.009299,...,-0.012364,-0.008707,-0.011149,-0.003092,-0.007956,-0.008111,-0.011135,-0.007594,-0.009499,-0.011958
x0_2.0,-0.020984,-0.040699,-0.012947,-0.015431,-0.040854,0.032834,-0.034033,-0.007877,1.000000,-0.007920,...,-0.010531,-0.007416,-0.009496,-0.002634,-0.006776,-0.006909,-0.009484,-0.006468,-0.008091,-0.010185
x0_3.0,-0.034021,-0.055531,-0.014096,-0.029904,-0.066852,0.054898,-0.052701,-0.009299,-0.007920,1.000000,...,-0.012432,-0.008754,-0.011210,-0.003109,-0.007999,-0.008156,-0.011196,-0.007636,-0.009551,-0.012024


In [162]:
features1 = model1_edit.iloc[:, 1:]
dep_variable = model1_edit['SalePrice']

In [163]:
model = sm.OLS(dep_variable, sm.add_constant(features1))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.430
Method:                 Least Squares   F-statistic:                     244.5
Date:                Tue, 03 Dec 2019   Prob (F-statistic):               0.00
Time:                        13:01:01   Log-Likelihood:            -4.4124e+05
No. Observations:               30011   AIC:                         8.827e+05
Df Residuals:                   29917   BIC:                         8.835e+05
Df Model:                          93                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.056e+05   4.55e+04      4.513      0.000    1.16e+05    2.95e+05
SqFtTotLiving   363.6444      6.276     57.938      0.000     351.342     375.946
SqFtLot           0.3655      0.066      5.511      0.000       0.235       0.495
Bedrooms      -6.645e+04   4688.834    -14.172      0.000   -7.56e+04   -5.73e+04
Bathrooms      1.001e+04   5975.297      1.675      0.094   -1703.349    2.17e+04
AgeOfBuilding -1083.8852    145.786     -7.435      0.000   -1369.633    -798.137
Garage         -128.6466     16.560     -7.769      0.000    -161.104     -96.189
x0_1.0         1.243e+05   5.58e+04      2.226      0.026    1.49e+04    2.34e+05
x0_2.0         9.818e+04   5.99e+04      1.639      0.101   -1.92e+04    2.16e+05
x0_3.0         4.685e+04   5.57e+04      0.841      0.400   -6.23e+04    1.56e+05
x0_4.0         -2.02e+04   5.72e+04     -0.353      0.724   -1.32e+05    9.19e+04
x0_6.0         1.825e+05   4.98e+04      3.665      0.000    8.49e+04     2.8e+05
x0_7.0         1.653e+05   5.23e+04      3.162      0.002    6.28e+04    2.68e+05
x0_8.0         1.412e+05   5.44e+04      2.597      0.009    3.46e+04    2.48e+05
x0_11.0        4.195e+05   5.29e+04      7.925      0.000    3.16e+05    5.23e+05
x0_12.0        6.083e+05   5.24e+04     11.617      0.000    5.06e+05    7.11e+05
x0_13.0        6.548e+05   5.33e+04     12.291      0.000     5.5e+05    7.59e+05
x0_14.0        8.356e+05   5.96e+04     14.027      0.000    7.19e+05    9.52e+05
x0_15.0        2.982e+05   5.08e+04      5.866      0.000    1.99e+05    3.98e+05
x0_16.0        2.834e+05   5.31e+04      5.339      0.000    1.79e+05    3.87e+05
x0_17.0        1.257e+05   5.35e+04      2.350      0.019    2.08e+04    2.31e+05
x0_18.0        6.808e+04   5.16e+04      1.320      0.187    -3.3e+04    1.69e+05
x0_19.0        3.098e+05   5.24e+04      5.909      0.000    2.07e+05    4.13e+05
x0_21.0        7.842e+04   5.22e+04      1.503      0.133   -2.39e+04    1.81e+05
x0_22.0       -6.288e+04    5.9e+04     -1.065      0.287   -1.79e+05    5.29e+04
x0_23.0       -1.043e+05   5.16e+04     -2.020      0.043   -2.05e+05   -3114.637
x0_24.0       -1.519e+05   5.15e+04     -2.948      0.003   -2.53e+05   -5.09e+04
x0_25.0       -8.056e+04    5.3e+04     -1.519      0.129   -1.85e+05    2.34e+04
x0_26.0       -2.103e+05   5.27e+04     -3.988      0.000   -3.14e+05   -1.07e+05
x0_27.0        3.553e+05   5.21e+04      6.817      0.000    2.53e+05    4.57e+05
x0_28.0       -1.598e+05   5.12e+04     -3.121      0.002    -2.6e+05   -5.94e+04
x0_29.0       -1.681e+05   5.64e+04     -2.984      0.003   -2.79e+05   -5.77e+04
x0_30.0       -1.317e+05   5.23e+04     -2.519      0.012   -2.34e+05   -2.92e+04
x0_31.0        2.627e+05   5.47e+04      4.805      0.000    1.56e+05     3.7e+05
x0_32.0       -2.807e+04   5.22e+04     -0.537   

In [ ]:
#Try looking for normal distributions of data
#Try normalizing or log transforming
#Try removing strongly colinear variables?

In [170]:
features2 = model1_edit.drop(['SalePrice', 'Bathrooms', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_18.0', 'x0_21.0',
                              'x0_22.0', 'x0_25.0', 'x0_32.0', 'x0_36.0', 'x0_38.0', 'x0_41.0', 'x0_49.0',
                              'x0_51.0', 'x0_53.0', 'x0_57.0', 'x0_60.0', 'x0_61.0', 'x0_69.0', 'x0_71.0', 'x0_73.0',
                              'x0_75.0', 'x0_77.0', 'x0_85.0', 'x0_90.0', 'x0_95.0', 'x0_96.0'], axis = 1)

In [171]:
model2 = sm.OLS(dep_variable, sm.add_constant(features2))
result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     341.8
Date:                Tue, 03 Dec 2019   Prob (F-statistic):               0.00
Time:                        13:23:53   Log-Likelihood:            -4.4130e+05
No. Observations:               30011   AIC:                         8.827e+05
Df Residuals:                   29944   BIC:                         8.833e+05
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.266e+05   1.62e+04     13.992      0.000    1.95e+05    2.58e+05
SqFtTotLiving   373.4336      5.272     70.832      0.000     363.100     383.767
SqFtLot           0.3646      0.065      5.576      0.000       0.236       0.493
Bedrooms      -6.703e+04   4550.441    -14.730      0.000   -7.59e+04   -5.81e+04
AgeOfBuilding -1274.3065    131.716     -9.675      0.000   -1532.476   -1016.137
Garage         -125.0427     16.111     -7.762      0.000    -156.620     -93.465
x0_1.0         1.168e+05   3.61e+04      3.233      0.001     4.6e+04    1.88e+05
x0_6.0         1.775e+05    2.6e+04      6.835      0.000    1.27e+05    2.28e+05
x0_7.0         1.613e+05   3.05e+04      5.289      0.000    1.02e+05    2.21e+05
x0_8.0         1.351e+05   3.39e+04      3.990      0.000    6.87e+04    2.01e+05
x0_11.0        4.142e+05   3.15e+04     13.131      0.000    3.52e+05    4.76e+05
x0_12.0         6.06e+05   3.06e+04     19.801      0.000    5.46e+05    6.66e+05
x0_13.0        6.525e+05   3.22e+04     20.289      0.000    5.89e+05    7.16e+05
x0_14.0        8.329e+05   4.18e+04     19.925      0.000    7.51e+05    9.15e+05
x0_15.0        2.949e+05    2.8e+04     10.548      0.000     2.4e+05     3.5e+05
x0_16.0        2.796e+05    3.2e+04      8.744      0.000    2.17e+05    3.42e+05
x0_17.0        1.205e+05   3.26e+04      3.695      0.000    5.66e+04    1.84e+05
x0_19.0        3.065e+05   3.08e+04      9.962      0.000    2.46e+05    3.67e+05
x0_23.0       -1.095e+05   2.93e+04     -3.738      0.000   -1.67e+05   -5.21e+04
x0_24.0       -1.575e+05   2.91e+04     -5.417      0.000   -2.15e+05   -1.01e+05
x0_26.0       -2.177e+05   3.09e+04     -7.036      0.000   -2.78e+05   -1.57e+05
x0_27.0        3.446e+05   2.98e+04     11.573      0.000    2.86e+05    4.03e+05
x0_28.0       -1.649e+05   2.85e+04     -5.786      0.000   -2.21e+05   -1.09e+05
x0_29.0        -1.77e+05   3.67e+04     -4.825      0.000   -2.49e+05   -1.05e+05
x0_30.0       -1.386e+05   3.02e+04     -4.590      0.000   -1.98e+05   -7.94e+04
x0_31.0        2.538e+05   3.41e+04      7.449      0.000    1.87e+05    3.21e+05
x0_33.0        2.217e+06   4.21e+04     52.603      0.000    2.13e+06     2.3e+06
x0_34.0        1.099e+06   3.22e+04     34.093      0.000    1.04e+06    1.16e+06
x0_35.0        9.899e+04   2.45e+04      4.038      0.000    5.09e+04    1.47e+05
x0_37.0        1.594e+05   2.69e+04      5.927      0.000    1.07e+05    2.12e+05
x0_39.0        2.803e+05   3.36e+04      8.332      0.000    2.14e+05    3.46e+05
x0_40.0       -2.694e+05   4.38e+04     -6.153      0.000   -3.55e+05   -1.84e+05
x0_42.0        3.998e+05   3.47e+04     11.515      0.000    3.32e+05    4.68e+05
x0_43.0         4.29e+05   3.19e+04     13.447      0.000    3.66e+05    4.91e+05
x0_44.0        3.511e+05   4.05e+04      8.678   

In [173]:
features2.corr()

,SqFtTotLiving,SqFtLot,Bedrooms,AgeOfBuilding,Garage,x0_1.0,x0_6.0,x0_7.0,x0_8.0,x0_11.0,...,x0_81.0,x0_82.0,x0_84.0,x0_86.0,x0_87.0,x0_88.0,x0_91.0,x0_92.0,x0_93.0,x0_94.0
SqFtTotLiving,1.000000,0.114822,0.626467,-0.374247,0.523475,0.014859,-0.087211,-0.060163,-0.021881,0.024608,...,-0.007550,-0.066566,0.005669,-0.022589,-0.021384,-0.050536,0.008513,0.112976,0.022995,0.001966
SqFtLot,0.114822,1.000000,-0.005922,0.002403,0.077098,0.001143,-0.029378,-0.021668,-0.018090,-0.022574,...,-0.014816,-0.025287,0.077984,-0.016625,-0.003258,-0.005169,-0.010405,-0.005507,-0.014213,0.117909
Bedrooms,0.626467,-0.005922,1.000000,-0.224351,0.321559,0.007384,-0.068196,-0.046240,-0.028618,-0.005100,...,-0.007734,-0.062717,-0.019436,0.017346,-0.007207,-0.020201,0.025598,0.054777,0.024964,-0.011521
AgeOfBuilding,-0.374247,0.002403,-0.224351,1.000000,-0.496575,0.037041,0.013561,0.075080,0.019636,0.050291,...,0.070889,0.086689,0.004976,-0.055574,-0.036677,0.024397,0.006057,-0.021558,-0.042981,-0.014835
Garage,0.523475,0.077098,0.321559,-0.496575,1.000000,0.005325,-0.092393,-0.072891,-0.029766,-0.047217,...,-0.053085,-0.092087,0.020479,0.042104,0.015177,-0.013820,0.038197,0.053689,0.038843,0.014553
x0_1.0,0.014859,0.001143,0.007384,0.037041,0.005325,1.000000,-0.013176,-0.011092,-0.009889,-0.010715,...,-0.008092,-0.010310,-0.007301,-0.012364,-0.008707,-0.011149,-0.007956,-0.008111,-0.011135,-0.007594
x0_6.0,-0.087211,-0.029378,-0.068196,0.013561,-0.092393,-0.013176,1.000000,-0.015803,-0.014089,-0.015266,...,-0.011529,-0.014689,-0.010402,-0.017615,-0.012405,-0.015884,-0.011335,-0.011557,-0.015864,-0.010820
x0_7.0,-0.060163,-0.021668,-0.046240,0.075080,-0.072891,-0.011092,-0.015803,1.000000,-0.011860,-0.012851,...,-0.009705,-0.012365,-0.008757,-0.014829,-0.010443,-0.013371,-0.009542,-0.009728,-0.013354,-0.009108
x0_8.0,-0.021881,-0.018090,-0.028618,0.019636,-0.029766,-0.009889,-0.014089,-0.011860,1.000000,-0.011457,...,-0.008653,-0.011024,-0.007807,-0.013220,-0.009310,-0.011921,-0.008507,-0.008673,-0.011906,-0.008120
x0_11.0,0.024608,-0.022574,-0.005100,0.050291,-0.047217,-0.010715,-0.015266,-0.012851,-0.011457,1.000000,...,-0.009376,-0.011945,-0.008459,-0.014325,-0.010088,-0.012917,-0.009218,-0.009398,-0.012901,-0.008799


In [180]:
from sklearn.preprocessing import StandardScaler